# Covid-19 Analysis
<a style="color:black" href="https://www.linkedin.com/in/ouassim-adnane/">Ouassim Adnane</a> May 2020</h5>

<img src="https://www.ecoactu.ma/wp-content/uploads/2020/04/coronavirus-1.jpg" style="width:70%;hight:100px;align:center">

<h2>This notebook tracks and Analyse the spread of the coronavirus(COVID-19) 🦠. </h2>

<h2>Content :</h2> 
<ul>
<li>Data preprocessing</li>
<li>Data visualization with Plotly and folium to track and analyze the spread of the virus </li>
<li>Showing what feature correlates with the number of fatalities (feature important and correlation matrix) </li>
<li>Trying the predict the number of confirmed cases from the number of fatalities and couple other feature to see if the real number of cases match that prediction</li> </ul>


# Acknowledgements
<a href="https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons">COVID-19 - Analysis, Visualization & Comparisons</a>

# Table of Content

1. [Data Description](#dd)
2. [Data preprocessing](#dp) <br>
    2.1 [Clustering the data](#cd)<br>
    2.2 [Checking for the optimal K in Kmeans Clustering](#co)<br>
    2.3 [Creating other useful columns](#cu)
3. [Data Visualization](#dv)<br>
    3.1 [Line Plots](#lp)<br>
    3.2 [Map](#mp)<br>
4. [Feature Importance](#fi)<br>
    4.1 [Correlation](#c)<br>
    4.2 [Feature importance with random forest](#fird)<br>
    4.3 [Checking the correlation between the Fatalities, Tourism and population size](#ccft)<br>
5. [Predicting the ConfirmCases From Fatalities](#pc)

# Data Description <a class="anchor" id="dd"></a>

<b>The COVID19 Global Forecasting (Week 4)</b> dataset thas daily level information on the number of affected cases and deaths from 2019 novel coronavirus.

#### Attribute Information: 
> 1. ID
> 2. Province_State 
> 3. Country_Region
> 4. resting blood pressure 
> 5. Date
> 6. ConfirmedCases
> 7. Fatalities


<b>The COVID 19 useful feature by country</b> dataset has information about countries and coronavirus

#### Attribute Information: 
> 1. Country_Region
> 2. Population_Size 
> 3. Tourism
> 4. Date_FirstFatality
> 5. Date_FirstConfirmedCase
> 6. Latitude
> 7. Longtitude
> 8. Mean_Age
> 9. Lockdown_Date
> 10. Lockdown_Type
> 11. Country_Code 3 digit code



<b>world-countries.json</b> Json file that contains information about countries geo Locations

# Data preprocessing <a class="anchor" id="dp"></a>

<h4>Import Libraries</h4>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import folium
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")
mms = MinMaxScaler()

<h4>Clustering the data</h4><a class="anchor" id="cd"></a>

In [ ]:
cluster_data = pd.read_csv("../input/covid19-useful-features-by-country/Countries_usefulFeatures.csv")
data = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
data2 = pd.read_csv("../input/covid19-global-forecasting-week-5/train.csv")
country_geo = "../input/world-countries/world-countries.json"
fd_i = pd.read_csv("../input/the-human-freedom-index/hfi_cc_2019.csv")

testing = pd.read_csv("../input/covid19testing/tested_worldwide.csv")


In [ ]:
max_d_test=testing.groupby(["Country_Region"]).agg({"daily_tested":"max"}).reset_index()

In [ ]:
r = {
     'Czech Republic': 'Czechia',
     'DR Congo': 'Congo (Brazzaville)',
     'Democratic Republic of the Congo': 'Congo (Kinshasa)',
     'Ivory Coast': "Cote d'Ivoire",
     'Palestine': 'West Bank and Gaza',
     'South Korea': 'Korea, South',
     'Taiwan': 'Taiwan*',
     'United States': 'US',
    }

In [ ]:
max_d_test.Country_Region=max_d_test.Country_Region.replace(to_replace=r)

In [ ]:
fd_i = fd_i[fd_i.year==2017]
fd_i = fd_i[["ISO_code","pf_score","hf_score"]]

In [ ]:
cluster_data.columns

In [ ]:
data.head()

In [ ]:
cluster_data.head()

In [ ]:
data_to_cluster = cluster_data.copy()

Clustering the data with those 5 columns 'Population_Size', 'Tourism', 'Date_FirstFatality','Date_FirstConfirmedCase', 'Latitude', 'Longtitude', 'Mean_Age'

In [ ]:
data_to_cluster = data_to_cluster[["Country_Region","Population_Size","Tourism","Date_FirstFatality","Date_FirstConfirmedCase","Latitude","Longtitude","Mean_Age"]]

In [ ]:
data_to_cluster.isnull().sum()

In [ ]:
data_to_cluster.Date_FirstFatality.fillna("2222-11-11",inplace=True)
data_to_cluster.Date_FirstFatality=pd.to_datetime(data_to_cluster.Date_FirstFatality)
data_to_cluster.Date_FirstFatality = data_to_cluster.Date_FirstFatality.astype(np.int64) // 10**9
data_to_cluster.Date_FirstConfirmedCase=pd.to_datetime(data_to_cluster.Date_FirstConfirmedCase)
data_to_cluster.Date_FirstConfirmedCase = data_to_cluster.Date_FirstConfirmedCase.astype(np.int64) // 10**9

In [ ]:
data_to_cluster.drop(["Country_Region"],axis=1,inplace=True)

In [ ]:
mms.fit(data_to_cluster)
data_transformed = mms.transform(data_to_cluster)

<h4>Checking for the optimal K in Kmeans Clustering</h4><a class="anchor" id="co"></a>

In [ ]:
Sum_of_squared_distances = []
K = range(1,90)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data_transformed)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Optimal K')
plt.show()


In [ ]:
km = KMeans(n_clusters=55,random_state=1995)
km = km.fit(data_transformed)

In [ ]:
cluster_data["cluster"] = km.labels_

In [ ]:
data.drop("Province_State",axis=1,inplace=True)

In [ ]:
data = pd.merge(data,cluster_data,on='Country_Region',how="left")

<h4>Creating other useful columns</h4><a class="anchor" id="cu"></a>

In [ ]:
data.columns

In [ ]:
temp_data = data.groupby(["Country_Region","Date"]).agg({"ConfirmedCases":"sum","Fatalities":"sum"}).reset_index()
temp_data.columns = ["Country_Region","Date","ConfirmedCases_i","Fatalities_i"]
data.drop_duplicates(["Country_Region","Date"],inplace=True)
data = pd.merge(data,temp_data,on=['Country_Region',"Date"],how="left")
data.drop(["ConfirmedCases","Fatalities"],axis=1,inplace=True)
data.rename(columns={"ConfirmedCases_i":"ConfirmedCases","Fatalities_i":"Fatalities","clusters":"cluster"},inplace=True)

Fatalities percentage 

In [ ]:
data.Date_FirstFatality.fillna("2222-11-11",inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data["Fatalities_rate"]=(data.Fatalities * 100) / data.ConfirmedCases

Difference between Lockdown Date and First Confirmed Case Date

In [ ]:
data["diff_FC_LD"]=(data.Lockdown_Date.astype('datetime64') - data.Date_FirstConfirmedCase.astype('datetime64'))

Difference between Lockdown Date and First Confirmed Fatality

In [ ]:
data["diff_FF_LD"]=(data.Lockdown_Date.astype('datetime64') - data.Date_FirstFatality.astype('datetime64'))

In [ ]:
tempd2=data2.loc[(data2.Target=="ConfirmedCases"),["Country_Region","Weight"]]

In [ ]:
tempd2["Weight_F"] =  data2.loc[(data2.Target=="Fatalities"),["Weight"]].values

In [ ]:
tempd2.columns = ["Country_Region","Weight_C","Weight_F"]

In [ ]:
tempd2=tempd2.drop_duplicates(["Country_Region"])

In [ ]:
data = pd.merge(data,tempd2[["Country_Region","Weight_C","Weight_F"]],on=['Country_Region'],how="left")

In [ ]:
fd_i.columns = ["Country_Code","pf_score","hf_score"]

In [ ]:
data = pd.merge(data,max_d_test[["Country_Region","daily_tested"]],on=['Country_Region'],how="left")

In [ ]:
data=pd.merge(data,fd_i[["Country_Code","pf_score","hf_score"]],on=['Country_Code'],how="left")

In [ ]:
data.daily_tested.fillna(data.loc[data.daily_tested>100,"daily_tested"].min(),inplace=True)

In [ ]:
data.hf_score.fillna(data.hf_score.median(),inplace=True)
data.pf_score.fillna(data.pf_score.median(),inplace=True)

# Data Visualization <a class="anchor" id="dv"></a>

<h3>Line Plots</h3>  <a class="anchor" id="lp"></a>

<h4>Checking the clusters</h4>  <a class="anchor" id="ctc"></a>

5 first Clusters

Line plot with Date and ConfirmedCases

In [ ]:
for i in data.cluster.unique()[:5]:
    test_df2= data[["Country_Region","Date","ConfirmedCases","cluster"]]
    test_df2=test_df2[ (test_df2.cluster==i) & (test_df2.Date > "2020-03-02") ]
    fig = px.line(test_df2, x="Date", y="ConfirmedCases", color='Country_Region')
    fig.show()

Line plot with Date and Fatalities

In [ ]:
for i in data.cluster.unique()[:5]:
    test_df2= data[["Country_Region","Date","Fatalities","cluster"]]
    test_df2=test_df2[ (test_df2.cluster==i) & (test_df2.Date > "2020-03-02") ]
    fig = px.line(test_df2, x="Date", y="Fatalities", color='Country_Region')
    fig.show()

<h4>Investigating the clusters</h4>  <a class="anchor" id="itc"></a>

In [ ]:
m_cluster=data[data.Country_Region == "Morocco"].cluster.unique()[0]
test_df2=data[ (data.cluster==m_cluster) & (data.Date > "2020-03-05") ]
fig = px.line(test_df2, x="Date", y="Fatalities", color='Country_Region')

for i in test_df2.Country_Region.unique():
    x=test_df2[test_df2.Country_Region == i].Lockdown_Date.unique()[0]
    y=test_df2[(test_df2.Date == x) & (test_df2.Country_Region == i)].Fatalities.values

    if (x is not np.NAN) & (len(y) >0) :
        fig.add_trace(go.Scatter(x=[x], y=[int(y)],
                        mode='markers',
                        name=i+'-lockdown-'+test_df2[test_df2.Country_Region == i].Lockdown_Type.unique()[0],
                        marker_symbol="x",
                        hoverinfo = 'text',
                        text=i+" Country lockdown Date "+ x,
                        marker = dict(
                                 size=15,
                                 color="black"
                             ),

                        ))

fig.show()

In [ ]:
m_cluster=data[data.Country_Region == "Spain"].cluster.unique()[0]
test_df2=data[ (data.cluster==m_cluster) & (data.Date > "2020-03-05") ]
fig = px.line(test_df2, x="Date", y="Fatalities", color='Country_Region')

for i in test_df2.Country_Region.unique():
    x=test_df2[test_df2.Country_Region == i].Lockdown_Date.unique()[0]
    y=test_df2[(test_df2.Date == x) & (test_df2.Country_Region == i)].Fatalities.values

    if (x is not np.NAN) & (len(y) >0) :
        fig.add_trace(go.Scatter(x=[x], y=[int(y)],
                        mode='markers',
                        name=i+'-lockdown-'+test_df2[test_df2.Country_Region == i].Lockdown_Type.unique()[0],
                        marker_symbol="x",
                        hoverinfo = 'text',
                        text=i+" Country lockdown Date "+ x,
                        marker = dict(
                                 size=15,
                                 color="black"
                             ),

                        ))

fig.show()

<h3>Maps</h3>  <a class="anchor" id="mp"></a>

<h4>Fatalities_rate</h4>  <a class="anchor" id="fr"></a>

In [ ]:
temp =data[(data.Date == max(data.Date)) ]

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=1.5)


for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

<h4>Mean_Age</h4>  <a class="anchor" id="ma"></a>

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=1.5)


for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='green', fill='green',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['Mean_Age']*2500)).add_to(m)

m

<h4>Fatalities_rate and Mean_Age</h4>  <a class="anchor" id="fm"></a>

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=8, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'Mean_Age'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="Mean_Age")

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

<h4>Fatalities_rate and Tourism</h4>  <a class="anchor" id="ft"></a>

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'Tourism'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="Tourism",
             threshold_scale=[14000.0,5000000, 20000000.0, 40000000.0, 60000000.0, 89322000.0]
             )

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(round(temp.iloc[i]['Fatalities_rate'],2))+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])+
                    '<li><bold>Tourism : '+str(temp.iloc[i]['Tourism'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

<h4>The Human Freedom Index</h4>

In [ ]:
temp.pf_score = temp.pf_score.astype("float")
temp.hf_score = temp.hf_score.astype("float")

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=8, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'pf_score'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="pf_score",
             )

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=8, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'hf_score'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="hf_score")

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

<h4>Number of tests per day by country</h4>

In [ ]:
temp.daily_tested = temp.daily_tested.astype("float")

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=8, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'daily_tested'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="daily_tested",
             threshold_scale=[0.0, 1000, 5000.0, 10000.0,50000.0,100000.0, 318720.0]
            )

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(temp.iloc[i]['Fatalities_rate'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])
        ,
        radius=int(temp.iloc[i]['ConfirmedCases']*1.1)).add_to(m)

m

In [ ]:
temp.to_csv("temp.csv",index=False)

<h4>Fatalities_rate and the difference between lockDown Date and FirstConfirmedCase</h4>  <a class="anchor" id="fdf"></a>

In [ ]:
temp.diff_FC_LD.replace({np.NAN:"62"},inplace=True)
temp.diff_FF_LD.replace({np.NAN:"42"},inplace=True)
temp.diff_FC_LD=temp.diff_FC_LD.astype(str)
temp.diff_FF_LD=temp.diff_FF_LD.astype(str)
temp.diff_FC_LD=temp.diff_FC_LD.str[:2]
temp.diff_FF_LD=temp.diff_FF_LD.str[:2]
temp.diff_FC_LD = temp.diff_FC_LD.astype(int)
temp.diff_FF_LD = temp.diff_FF_LD.astype(int)

In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=10, zoom_start=1.5)

m.choropleth(geo_data=country_geo, data=temp,
             columns=['Country_Code', 'diff_FC_LD'],
             key_on='feature.id',
             # 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.
             fill_color='RdPu', fill_opacity=0.7, line_opacity=0.1,
             legend_name="FirstCaseConfirmed_diff_lockdown"
             )

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Latitude'], temp.iloc[i]['Longtitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country_Region'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['ConfirmedCases'])+
                    '<li><bold>Death_rate : '+str(round(temp.iloc[i]['Fatalities_rate'],2))+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Fatalities'])+
                    '<li><bold>lockdown date : '+str(temp.iloc[i]['Lockdown_Date'])+
                    '<li><bold>first case date : '+str(temp.iloc[i]['Date_FirstConfirmedCase'])+
                    '<li><bold>mean age : '+str(temp.iloc[i]['Mean_Age'])+
                    '<li><bold>Tourism : '+str(temp.iloc[i]['Tourism'])
        ,
        radius=int(temp.iloc[i]['Fatalities_rate']*10000)).add_to(m)

m

# Feature Importance <a class="anchor" id="fi"></a>

<h4>Correlation</h4><a class="anchor" id="c"></a>

In [ ]:
data.pf_score = data.pf_score.astype("float")
data.hf_score = data.hf_score.astype("float")

In [ ]:
import seaborn as sns
corr = data.drop(["Id"],axis=1).corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,mask=mask,cmap='summer_r',vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": 0.5})


<h4>Feature importance with random forest</h4> <a class="anchor" id="fird"></a>

In [ ]:
def feature_imp(df,model):
    fi = pd.DataFrame()
    fi["feature"] = df.columns
    fi["importance"] = model.feature_importances_
    return fi.sort_values(by="importance", ascending=False)

In [ ]:
train = data.copy()

In [ ]:
train.Date_FirstFatality.fillna("2222-11-11",inplace=True)
train.Date_FirstFatality=pd.to_datetime(train.Date_FirstFatality)
train.Date_FirstFatality = train.Date_FirstFatality.astype(np.int64) // 10**9

train.Date_FirstConfirmedCase=pd.to_datetime(train.Date_FirstConfirmedCase)
train.Date_FirstConfirmedCase = train.Date_FirstConfirmedCase.astype(np.int64) // 10**9

train.Date=pd.to_datetime(train.Date)
train.Date = train.Date.astype(np.int64) // 10**9

train.Lockdown_Date.fillna("2111-12-12",inplace=True)
train.Lockdown_Date=pd.to_datetime(train.Lockdown_Date)
train.Lockdown_Date = train.Lockdown_Date.astype(np.int64) // 10**9


train.Lockdown_Type = train.Lockdown_Type.astype("category")
train.Lockdown_Type = train.Lockdown_Type.cat.codes

train.diff_FC_LD.replace({np.NAN:"62"},inplace=True)
train.diff_FF_LD.replace({np.NAN:"42"},inplace=True)
train.diff_FC_LD=train.diff_FC_LD.astype(str)
train.diff_FF_LD=train.diff_FF_LD.astype(str)
train.diff_FC_LD=train.diff_FC_LD.str[:2]
train.diff_FF_LD=train.diff_FF_LD.str[:2]
train.diff_FC_LD = train.diff_FC_LD.astype(int)
train.diff_FF_LD = train.diff_FF_LD.astype(int)

In [ ]:
train = train[train.Date == max(train.Date)]

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000,
                           n_jobs = -1,
                           random_state = 141)
rf.fit(train.drop(['Country_Region',"Id","Fatalities","ConfirmedCases","Fatalities_rate","Date","Country_Code","Date_FirstFatality","Date_FirstConfirmedCase","Lockdown_Date"],axis=1), train["Fatalities"])

In [ ]:
fimp = feature_imp(train.drop(['Country_Region',"Id","Fatalities","ConfirmedCases","Fatalities_rate","Date","Country_Code","Date_FirstFatality","Date_FirstConfirmedCase","Lockdown_Date"],axis=1),rf)

In [ ]:
feature_imp(train.drop(['Country_Region',"Id","Fatalities","ConfirmedCases","Fatalities_rate","Date","Country_Code","Date_FirstFatality","Date_FirstConfirmedCase","Lockdown_Date"],axis=1),rf).plot('feature', 'importance', 'barh', figsize=(12,7), legend=False)

<h4>Checking the correlation between the Fatalities, Tourism and population size</h4> <a class="anchor" id="ccft"></a>

In [ ]:
fig = px.scatter(data[(data.Tourism>100000) & (data.Date == max(data.Date)) ],
                 x='Tourism', y='Fatalities', color='Country_Region', size='Mean_Age', height=700,
                 text='Country_Region',log_x=True,log_y=True)
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.scatter(data[(data.Population_Size>100000) & (data.Date == max(data.Date)) ],
                 x='Population_Size', y='Fatalities', color='Country_Region', size='Mean_Age', height=700,
                 text='Country_Region',log_x=True,log_y=True)
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.scatter(data[(data.Population_Size>100000) & (data.Date == max(data.Date)) ],
                 x='Weight_F', y='Fatalities', color='Country_Region', size='Mean_Age', height=700,
                 text='Country_Region',log_x=True,log_y=True)
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.scatter(data[(data.daily_tested>200) & (data.Date == max(data.Date)) ],
                 x='daily_tested', y='Fatalities', color='Country_Region', size='Mean_Age', height=700,
                 text='Country_Region',log_x=True,log_y=True)
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# Predicting the ConfirmCases From Fatalities   <a class="anchor" id="pc"></a>

In [ ]:
data.sort_values(["Country_Region"],inplace=True)

In [ ]:
def predict_ConfirmedCases(test_country):
    tmp_data=data[(data.Date == max(data.Date))]
    test_data=tmp_data[tmp_data.Country_Region == test_country]
    train_data = tmp_data[(tmp_data.Country_Region !=test_country) & (tmp_data.Fatalities_rate >= 2) & (tmp_data.Fatalities_rate <= 6)]
    #print(test_data)
    
    mms.fit(train_data[['Fatalities',"Population_Size","Tourism"]])
    data_transformed = mms.transform(train_data[['Fatalities',"Population_Size","Tourism"]])
    
    rg = Ridge(alpha=0.00001)
    rg.fit(data_transformed,train_data["ConfirmedCases"])
   
    predictions=rg.predict(mms.transform(test_data[['Fatalities',"Population_Size","Tourism"]]))
    return pd.DataFrame([test_country,predictions[0]]).transpose()

In [ ]:
tmp_df = pd.DataFrame([data.Country_Region.unique()]).T

In [ ]:
tmp_df.columns =["Country_Region"]

In [ ]:
for i in data.Country_Region.unique():
    d=predict_ConfirmedCases(i)
    tmp_df.loc[tmp_df.Country_Region == i,"predicted_C"] = predict_ConfirmedCases(i)[1].values[0]

In [ ]:
tmp_df=pd.merge(tmp_df, data.loc[(data.Date == max(data.Date)),["ConfirmedCases","Country_Region"]],how="left",on="Country_Region")

In [ ]:
tmp_df.predicted_C = tmp_df.predicted_C.apply(lambda x: int(round(x,2)))

In [ ]:
tmp_df[tmp_df.Country_Region.isin(["Morocco","Egypt","Algeria","Tunisia","France","Spain","Italy","Korea, South"])]

<h2>Thanks for reading </h2>
<h2>If you found this helpful a upvote would be very much appreciated :-)</h2>